In [1]:
import json
from pathlib import Path

data = json.load(open(f"{Path.home()}/data/vivid_video_instruct.json"))

In [2]:
sorted(data, key=lambda x: x["video"])[0]

{'id': '01gE8ErrcJ4.0',
 'video': 'shard_0/01gE8ErrcJ4.0.mp4',
 'conversations': [{'from': 'human',
   'value': '<video>\nMô tả cụ thể các hành động, nhân vật và bối cảnh xuất hiện trong video.'},
  {'from': 'gpt',
   'value': 'Video bắt đầu với hình ảnh một phổi được mô tả bằng hình ảnh 3D. Phổi được mô tả với nhiều màu sắc khác nhau. Sau đó, video chuyển sang hình ảnh mô tả cấu trúc của phổi với nhiều túi khí nhỏ.'}],
 'path': 'Vividbot/vivid-video-instruct/videos'}

In [6]:
longs = [x for x in data if len(x["conversations"]) > 2 * 3]
shorts = [x for x in data if len(x["conversations"]) <= 2 * 3]

In [33]:
superlongs = []

for i in range(0, 10000):
  # total word count incrementally of all conversations, i.e. 1st, 1st+2nd, 1st+2nd+3rd, etc.

  total_word_count = sum(
    [
      sum([len(x["value"].split(" ")) for x in data[i]["conversations"][:j]])
      for j in range(1, len(data[i]["conversations"]) + 1)
    ]
  )
  if total_word_count > 5000:
    superlongs.append((i, len(data[i]["conversations"]), data[i]["conversations"]))

In [7]:
sorted_data = sorted(
  data,
  key=lambda x: sum(
    [
      sum([len(x["value"].split(" ")) for x in x["conversations"][:j]])
      for j in range(1, len(x["conversations"]) + 1)
    ]
  ),
  reverse=True,
)

In [8]:
len(sorted_data)

97448

In [9]:
sorted_data[0]

{'id': 'RXk4fg4rVY0.7',
 'video': 'shard_57/RXk4fg4rVY0.7.mp4',
 'conversations': [{'from': 'human',
   'value': '<video>\nĐối tượng và sự kiện trong đoạn video là gì?'},
  {'from': 'gpt',
   'value': 'Đoạn video bắt đầu với một gia đình gồm bố mẹ và hai con nhỏ đang ngồi trên bờ biển, nhìn về phía mặt trời đang lặn. Sau đó, video chuyển sang hình ảnh một người phụ nữ đang ngồi trên ghế sofa với vẻ mặt buồn bã. Tiếp theo, video hiển thị hai người đang nắm tay nhau với nền là hoàng hôn. Cuối cùng, video kết thúc với một gia đình gồm bố mẹ và con trai đang ngồi trên một chiếc thuyền nhỏ, nhìn về phía mặt trời đang lặn.'},
  {'from': 'human',
   'value': 'Theo video, những cảnh quan và hoàn cảnh nào được thể hiện?'},
  {'from': 'gpt',
   'value': 'Trong đoạn video, các cảnh quan chính được thể hiện bao gồm: bờ biển với gia đình ngắm mặt trời lặn, người phụ nữ ngồi trên ghế sofa với vẻ mặt buồn bã, hai người đang nắm tay nhau với nền là hoàng hôn, và một gia đình ngồi trên thuyền nhỏ nhìn 

In [34]:
len(superlongs)

1

In [35]:
superlongs

[(8471,
  20,
  [{'from': 'human', 'value': 'Video này nói về chủ đề gì?\n<video>'},
   {'from': 'gpt',
    'value': 'Video này thảo luận về lý thuyết rằng các kim tự tháp ở Giza được xây dựng như một nguồn năng lượng khổng lồ, sử dụng các thuộc tính của vật liệu chúng được xây dựng để tạo ra và truyền tải năng lượng.'},
   {'from': 'human', 'value': 'Kim tự tháp Giza được căn chỉnh với cái gì?'},
   {'from': 'gpt',
    'value': 'Video bắt đầu bằng việc nêu bật sự căn chỉnh chính xác của ba kim tự tháp với vành đai của chòm sao Orion, gợi ý về mối liên hệ giữa các kim tự tháp và các thiên thể.'},
   {'from': 'human',
    'value': 'Tại sao đá vôi trắng Tura lại quan trọng trong việc xây dựng kim tự tháp?'},
   {'from': 'gpt',
    'value': 'Đá vôi trắng Tura được sử dụng cho lớp vỏ ngoài của các kim tự tháp, đóng vai trò như một chất cách điện, ngăn ngừa sự mất năng lượng từ cấu trúc.'},
   {'from': 'human',
    'value': 'Cấu trúc bên trong của kim tự tháp bao gồm những vật liệu gì?'},
 

In [110]:
# new_data = []

# for d in data:
#   d_copy = d.copy()
#   new_conv = []
#   for i in range(0, len(d["conversations"]), 2):
#     conv = d["conversations"][i : i + 2]
#     conv[0]["value"] = conv[0]["value"].strip()
#     conv[1]["value"] = conv[1]["value"].strip()
#     if len(conv[1]["value"].split(" ")) > 10:
#       new_conv.extend(conv)
#   if len(new_conv) > 0:
#     d_copy["conversations"] = new_conv[:6]
#     new_data.append(d_copy)

In [3]:
new_data = []
superlongs = []

for d in data:
  d_copy = d.copy()
  new_conv = []
  for i in range(0, len(d["conversations"]), 2):
    conv = d["conversations"][i : i + 2]
    conv[0]["value"] = conv[0]["value"].strip()
    conv[1]["value"] = conv[1]["value"].strip()
    if (
      len(conv[1]["value"].split(" ")) > 10
      and len(conv[1]["value"].split(" ")) < 120
      and len(conv[0]["value"].split(" ")) > 3
      and len(conv[0]["value"].split(" ")) < 60
    ):
      new_conv.extend(conv)
  if len(new_conv) > 0:
    d_copy["conversations"] = []
    total_word_count = 0
    for i in range(0, len(new_conv), 2):
      try:
        total_word_count += total_word_count + len(new_conv[i]["value"].split(" "))
        if total_word_count < 1200:
          d_copy["conversations"].extend(new_conv[i : i + 2])
        else:
          superlongs.append(d)
          break
        total_word_count += len(new_conv[i + 1]["value"].split(" "))
      except:
        pass
    new_data.append(d_copy)

In [4]:
len(new_data)

97448

In [52]:
new_data[0]

{'id': '01gE8ErrcJ4.0',
 'video': 'shard_0/01gE8ErrcJ4.0.mp4',
 'conversations': [{'from': 'human',
   'value': '<video>\nMô tả cụ thể các hành động, nhân vật và bối cảnh xuất hiện trong video.'},
  {'from': 'gpt',
   'value': 'Video bắt đầu với hình ảnh một phổi được mô tả bằng hình ảnh 3D. Phổi được mô tả với nhiều màu sắc khác nhau. Sau đó, video chuyển sang hình ảnh mô tả cấu trúc của phổi với nhiều túi khí nhỏ.'}]}

In [5]:
sorted_new_data = sorted(
  new_data,
  key=lambda x: sum(
    [
      sum([len(x["value"].split(" ")) for x in x["conversations"][:j]])
      for j in range(1, len(x["conversations"]) + 1)
    ]
  ),
  reverse=True,
)

In [6]:
sorted_new_data[0]

{'id': 'RXk4fg4rVY0.7',
 'video': 'shard_57/RXk4fg4rVY0.7.mp4',
 'conversations': [{'from': 'human',
   'value': '<video>\nĐối tượng và sự kiện trong đoạn video là gì?'},
  {'from': 'gpt',
   'value': 'Đoạn video bắt đầu với một gia đình gồm bố mẹ và hai con nhỏ đang ngồi trên bờ biển, nhìn về phía mặt trời đang lặn. Sau đó, video chuyển sang hình ảnh một người phụ nữ đang ngồi trên ghế sofa với vẻ mặt buồn bã. Tiếp theo, video hiển thị hai người đang nắm tay nhau với nền là hoàng hôn. Cuối cùng, video kết thúc với một gia đình gồm bố mẹ và con trai đang ngồi trên một chiếc thuyền nhỏ, nhìn về phía mặt trời đang lặn.'},
  {'from': 'human',
   'value': 'Theo video, những cảnh quan và hoàn cảnh nào được thể hiện?'},
  {'from': 'gpt',
   'value': 'Trong đoạn video, các cảnh quan chính được thể hiện bao gồm: bờ biển với gia đình ngắm mặt trời lặn, người phụ nữ ngồi trên ghế sofa với vẻ mặt buồn bã, hai người đang nắm tay nhau với nền là hoàng hôn, và một gia đình ngồi trên thuyền nhỏ nhìn 

In [63]:
# new_data = [
#   {**x, "path": ""}
#   for x in sorted_new_data
#   if len(x["conversations"][1]["value"].split(" ")) > 10
# ]

In [64]:
new_data = sorted(new_data, key=lambda x: x["video"])

In [65]:
len(new_data)

97448

In [66]:
new_data = [{**d, "path": "Vividbot/vivid-video-instruct/videos"} for d in new_data]

In [67]:
with open(f"{Path.home()}/data/vivid_video_instruct.json", "w") as f:
  json.dump(new_data, f, indent=2, ensure_ascii=False)

In [66]:
sorted_new_data = sorted(
  new_data,
  key=lambda x: sum(
    [
      sum([len(x["value"].split(" ")) for x in x["conversations"][:j]])
      for j in range(1, len(x["conversations"]) + 1)
    ]
  ),
  reverse=True,
)

In [74]:
i = 1

total_word_count = sum(
  [
    sum([len(x["value"].split(" ")) for x in sorted_new_data[i]["conversations"][:j]])
    for j in range(1, len(sorted_new_data[i]["conversations"]) + 1)
  ]
)
total_word_count, sorted_new_data[i]

(975,
 {'id': 'KRsrPnVGVkE.5',
  'video': 'shard_59/KRsrPnVGVkE.5.mp4',
  'conversations': [{'from': 'human',
    'value': '<video>\nHình ảnh minh họa cho podcast này là gì?'},
   {'from': 'gpt',
    'value': 'Hình ảnh là một căn phòng với một chiếc ghế sofa màu xanh lá cây có đệm sọc trắng và xanh lá cây, trên bàn cạnh ghế sofa có một chậu cây nhỏ, bên cạnh bàn là một chiếc bàn nhỏ khác có một chậu cây nhỏ khác, ở góc trên bên phải của hình ảnh là một chiếc điện thoại thông minh màu xanh lá cây với một biểu tượng phát màu trắng, trên màn hình điện thoại là một hình ảnh minh họa cho một podcast với một cô gái đang đọc sách, ở góc trên bên trái của hình ảnh là một dòng chữ màu xanh dương "Tối giản" và một dòng chữ màu xanh dương "Tối ưu" ở bên dưới, ở giữa hình ảnh là một dòng chữ màu xanh dương "Lối sống cuộc đời" với các chữ cái được trang trí bằng các hình ngôi sao màu trắng, ở góc dưới bên phải của hình ảnh là một dòng chữ màu xanh dương "RADIOCHUALANHTRAITIM".'},
   {'from': 'human

In [1]:
import json
from pathlib import Path

d2 = json.load(open(f"{Path.home()}/data/vivid_video_instruct.json"))

In [2]:
len(d2)

97448

In [4]:
d2[-1]

{'id': 'z3X29hqSTGQ.9',
 'video': 'shard_99/z3X29hqSTGQ.9.mp4',
 'conversations': [{'from': 'human',
   'value': 'Khi nào vụ tấn công tượng đài xảy ra?'},
  {'from': 'gpt',
   'value': 'Khoảng 3 giờ sáng ngày 10 tháng 6 theo giờ địa phương'}],
 'path': 'Vividbot/vivid-video-instruct/videos'}